# Model Selection

I'll train a logistic regression model using K-Folds Cross Validation.  

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

# load processed data
train = pd.read_csv('../data/processed/train.csv')
test = pd.read_csv('../data/processed/test.csv')

In [22]:
def fit(model, train, test, shuffle, n_folds, save, random_state):
    """
    Trains and tests a model
    
    inputs:
    
    model - the model to train
    train - training data
    test - test data
    shuffle - either True/False.  Shuffle if True.
    n_folds - number of folds
    save - if True, save predictions.
    """
    # dict to store trained models
    models = {}
    
    y_train = train['status_group']
    X_train = train.drop(['status_group'], axis = 1)
    labels = train['status_group'].unique()[:1]
    
    for label in labels:
        # instantiate new model
        model = model
        
        y_train = y_train == label
        y_train = pd.factorize(y_train, sort=True)[0]

        # declare a KFold instance
        kfold = KFold(n_splits = n_folds, random_state = random_state)
        result = []
        
        for i, (train, val) in enumerate(kfold.split(train)):
            # get smaller training set and create validation set
            X_train_mini, X_val = X_train.iloc[train], X_train.iloc[val]
            y_train_mini, y_val = y_train[train], y_train[val]
            model.fit(X_train_mini.values, y_train_mini)
            predictions = model.predict_proba(X_val.values)
            print(predictions[:5])
            #accuracy = accuracy_score(y_val, predictions)
            #result.append(accuracy)
            
        avg_acc = np.mean(result)            

In [ ]:
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import MinMaxScaler
def normalize(df, col):
    """Normalizes a column"""
    X = df[col].reshape(-1, 1)
    mms = MinMaxScaler().fit(X = X)
    return mms.transform(X = X)

train['latitude'] = normalize(train, 'latitude')
train['longitude'] = normalize(train, 'longitude')
test['latitude'] = normalize(test, 'latitude')
test['longitude'] = normalize(test, 'longitude')

lr = LogisticRegression()
fit(model = lr, train = train, test = test, shuffle = False, n_folds = 5, save = False, random_state = 5)

In [27]:
pd.options.display.max_columns = 200
np.inf(train['status_group'])

TypeError: 'float' object is not callable